In [21]:
import requests
import pandas as pd
import sqlite3

from bs4 import BeautifulSoup
from time import time
from datetime import datetime

In [22]:
base_url = 'https://pastebin.com'
url = 'https://pastebin.com/archive'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find('table', class_="maintable")

url_list = []

for link in table.select('tbody tr td span ~ a'):
  item_link = base_url + link.get('href')
  raw_link = base_url + link.get('href')

  response = requests.get(raw_link)
  all_text = response.text

  text_stripped = all_text.split()
  word_count = len(text_stripped)

  unique_text = set(text_stripped)
  word_diff_count = len(unique_text)
   
  response = requests.get(base_url + link.get('href'))
  soup = BeautifulSoup(response.content, 'html.parser')

  search = soup.select_one('div.highlighted-code > div.top-buttons > div.left > a')
  syntax = search.text
  
  now = time()
  now_time = datetime.utcfromtimestamp(now).strftime('%Y-%m-%d %H:%M:%S')

  url_list.append([item_link, word_count, word_diff_count, syntax ,now_time])


df = pd.DataFrame(data = url_list, columns = ['page_link', 'word_count', 'word_diff_count', 'syntax' ,'pickup_time'])
df.to_csv('data_scraping.csv', index = False)

In [24]:
# CRIA CONEXÃO COM O BANCO DE DADOS SQLite3
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn

conn = create_connection('scraping.db')

# CONVERTE O DATAFRAME EM UMA TABELA DO SQLITE3
df.to_sql(name = 'pastebin', index = False, con = conn, if_exists = 'append') # if_exists ('fail', 'replace' or 'append')

50

In [25]:
# FECHA CONEXÃO COM BANCO
conn.close()